<a href="https://colab.research.google.com/github/HakureiPOI/Comp_Arch/blob/lab4-5/Arch_Lab_4_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***HakureiPOI*** *Arch_Lab_4-5*

---
## *环境准备*

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
!nvidia-smi

Thu Dec 26 05:55:33 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
!git clone https://github.com/HakureiPOI/Comp_Arch.git

Cloning into 'Comp_Arch'...
remote: Enumerating objects: 987, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 987 (delta 35), reused 44 (delta 7), pack-reused 900 (from 2)
Receiving objects: 100% (987/987), 62.12 MiB | 17.01 MiB/s, done.
Resolving deltas: 100% (604/604), done.


In [4]:
%cd Comp_Arch

/content/Comp_Arch


In [5]:
!git branch -a

* main
  remotes/origin/HEAD -> origin/main
  remotes/origin/lab1
  remotes/origin/lab2
  remotes/origin/lab3
  remotes/origin/lab4-5
  remotes/origin/lab6
  remotes/origin/main


In [6]:
!git switch lab4-5

Branch 'lab4-5' set up to track remote branch 'lab4-5' from 'origin'.
Switched to a new branch 'lab4-5'


In [7]:
%cd lab4-5

/content/Comp_Arch/lab4-5


---
## *Lab4*


### *实现 CUDA 矩阵乘法核函数*

* Step 1: 计算行和列索引
在核函数中，首先需要计算每个线程负责计算的输出矩阵 `P` 的元素位置。这通过 `blockIdx` 和 `threadIdx` 来实现，它们分别表示线程块和线程在网格中的位置。

```cuda
int row = blockIdx.y * blockDim.y + threadIdx.y;
int col = blockIdx.x * blockDim.x + threadIdx.x;
```

这里，`blockDim` 定义了每个线程块的尺寸，`blockIdx` 和 `threadIdx` 用于计算每个线程的全局位置。

* Step 2: 确保线程在矩阵边界内
在进行计算之前，需要确保线程计算的位置在矩阵的边界内，以避免越界访问。

```cuda
if (row < width && col < width) {
    float pValue = 0.0;
```

* Step 3: 计算矩阵乘法的单个元素
每个线程计算矩阵 `P` 的一个元素，这是通过遍历矩阵 `M` 的列（或矩阵 `N` 的行）来实现的。

```cuda
for (int k = 0; k < width; ++k) {
    pValue += d_M[row * width + k] * d_N[k * width + col];
}
```

这里，`d_M` 和 `d_N` 分别是输入矩阵 `M` 和 `N` 在设备上的存储，`width` 是矩阵的宽度（对于方阵来说，宽度等于高度）。

* Step 4: 存储计算结果
计算完成后，将结果存储到输出矩阵 `P` 的相应位置。

```cuda
d_P[row * width + col] = pValue;
```


具体的代码实现如下


```
__global__ void MatrixMulKernel(float *d_M, float *d_N, float *d_P, int width) {
  // Calculate the row index of the P element and M
  int row = blockIdx.y * blockDim.y + threadIdx.y;

  // Calculate the column index of the P element and N
  int col = blockIdx.x * blockDim.x + threadIdx.x;

  // Ensure the thread is within bounds
  if (row < width && col < width) {
    float pValue = 0.0;

    // Each thread computes one element of the matrix
    for (int k = 0; k < width; ++k) {
      pValue += d_M[row * width + k] * d_N[k * width + col];
    }

    // Store the computed value into the output matrix
    d_P[row * width + col] = pValue;
  }
}
```



### *尝试编译*

In [8]:
!nvcc -arch=compute_75 -L/usr/local/cuda/lib64 -lcublas ./matrix_mul_lab4.cu

./matrix_mul_lab4.cu(198): warning #177-D: variable "alpha" was declared but never referenced
    const float alpha = 1.0f;
                ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

./matrix_mul_lab4.cu(199): warning #177-D: variable "beta" was declared but never referenced
    const float beta = 0.0f;
                ^

./matrix_mul_lab4.cu(18): warning #177-D: variable "BLOCK_SIZE" was declared but never referenced
  const int BLOCK_SIZE = TILE_WIDTH;
            ^



### *脚本测试*

In [9]:
!bash testbench_lab4.sh

Compiling with TILE_SIZE=16
Running tests with TILE_SIZE=16
Matrix size: 128
Kernel Elpased Time: 0.061 ms
Performance= 69.28 GFlop/s, Time= 0.061 msec, Size= 4194304 Ops
Computing result using host CPU...done.
Listing first 100 Differences > 0.000010...
 
  Total Errors = 0
--------------------------------

Matrix size: 256
Kernel Elpased Time: 0.176 ms
Performance= 190.14 GFlop/s, Time= 0.176 msec, Size= 33554432 Ops
Computing result using host CPU...done.
Listing first 100 Differences > 0.000010...
 
  Total Errors = 0
--------------------------------

Matrix size: 512
Kernel Elpased Time: 1.149 ms
Performance= 233.61 GFlop/s, Time= 1.149 msec, Size= 268435456 Ops
Computing result using host CPU...done.
Listing first 100 Differences > 0.000010...
 
  Total Errors = 0
--------------------------------

Matrix size: 1024
Kernel Elpased Time: 9.124 ms
Performance= 235.37 GFlop/s, Time= 9.124 msec, Size= 2147483648 Ops
Computing result using host CPU...done.
Listing first 100 Differences

### *测试结果分析*

***1. TILE_SIZE对性能的影响***

- **TILE_SIZE=16**：
  - 在较小的矩阵大小（128, 256）时，性能相对较低。
  - 随着矩阵大小的增加，性能逐渐提升，但提升幅度不如TILE_SIZE=32和TILE_SIZE=64。

- **TILE_SIZE=32**：
  - 在所有测试的矩阵大小中，性能普遍高于TILE_SIZE=16。
  - 在2048大小的矩阵中，性能达到最高，为306.83 GFlop/s。

- **TILE_SIZE=64**：
  - 性能略低于TILE_SIZE=32，但在某些情况下（如512和1024大小的矩阵）与TILE_SIZE=32相近。
  - 在2048大小的矩阵中，性能为297.75 GFlop/s，略低于TILE_SIZE=32。

***2. 矩阵大小对性能的影响***

- **小矩阵（128, 256）**：
  - 所有TILE_SIZE设置下，小矩阵的性能相对较低。
  - 这可能是因为小矩阵的数据量不足以充分利用并行计算资源。

- **中等矩阵（512, 1024）**：
  - 性能显著提升，尤其是在TILE_SIZE=32和TILE_SIZE=64时。
  - 这表明中等大小的矩阵能更好地利用并行计算资源。

- **大矩阵（2048）**：
  - 性能达到最高，尤其是在TILE_SIZE=32时。
  - 大矩阵能充分利用并行计算资源，尤其是在较大的TILE_SIZE设置下。

***3. 性能趋势和规律***

- **性能提升与矩阵大小的关系**：
  - 随着矩阵大小的增加，性能提升，尤其是在较大的TILE_SIZE设置下。
  - 这表明并行计算核能够有效地扩展以处理更大的数据集。

- **TILE_SIZE的最优值**：
  - TILE_SIZE=32在大多数情况下提供了最佳性能，尤其是在处理大矩阵时。
  - 这可能是因为较大的TILE_SIZE能更有效地利用缓存和并行处理能力。

- **错误检查**：
  - 在所有测试中，总错误数为0，表明核计算的结果与主机CPU的结果一致，验证了核的正确性。

***4. 结论***

- **最优TILE_SIZE选择**：TILE_SIZE=32在大多数情况下提供了最佳性能，尤其是在处理大矩阵时。
- **矩阵大小的影响**：随着矩阵大小的增加，性能提升，表明并行计算核能够有效地扩展以处理更大的数据集。
- **性能一致性**：核的计算结果与主机CPU的结果一致，验证了核的正确性。


---
## Lab5

### 尝试编译

In [10]:
!nvcc -arch=compute_75 -L/usr/local/cuda/lib64 -lcublas ./matrix_mul_lab5.cu

./matrix_mul_lab5.cu(176): warning #177-D: variable "alpha" was declared but never referenced
      const float alpha = 1.0f;
                  ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

./matrix_mul_lab5.cu(177): warning #177-D: variable "beta" was declared but never referenced
      const float beta = 0.0f;
                  ^



### 脚本测试

In [11]:
!bash testbench_lab5.sh

---****** Testing with cuBLAS ******---

Testing with TILE_WIDTH=16
use cublas with TILE_WIDTH=16

matrix_size: 128, TILE_WIDTH: 16
Kernel Elapsed Time: 0.049 ms
Performance= 84.92 GFlop/s
--------------------------------

matrix_size: 256, TILE_WIDTH: 16
Kernel Elapsed Time: 0.165 ms
Performance= 203.83 GFlop/s
--------------------------------

matrix_size: 512, TILE_WIDTH: 16
Kernel Elapsed Time: 1.011 ms
Performance= 265.53 GFlop/s
--------------------------------

matrix_size: 1024, TILE_WIDTH: 16
Kernel Elapsed Time: 7.813 ms
Performance= 274.86 GFlop/s
--------------------------------

matrix_size: 2048, TILE_WIDTH: 16
Kernel Elapsed Time: 49.061 ms
Performance= 350.17 GFlop/s
--------------------------------

Testing with TILE_WIDTH=32
use cublas with TILE_WIDTH=32

matrix_size: 128, TILE_WIDTH: 32
Kernel Elapsed Time: 0.062 ms
Performance= 68.12 GFlop/s
--------------------------------

matrix_size: 256, TILE_WIDTH: 32
Kernel Elapsed Time: 0.170 ms
Performance= 197.29 GFlop/s
-